In [13]:
!pip install torcheval

import os
import numpy as np 
import pandas as pd 
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset, random_split
from torcheval.metrics import R2Score
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

In [ ]:
# Dataset to be retrived , only if not using kaggle and paths must be manipulated accordingly
"""import kagglehub
kagglehub.login()
playground_series_s5e5_path = kagglehub.competition_download('playground-series-s5e5')
print('Data source import complete.')"""

# Data Processing

In [14]:
train=pd.read_csv("/kaggle/input/playground-series-s5e5/train.csv")
test=pd.read_csv("/kaggle/input/playground-series-s5e5/test.csv")
train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [15]:
test.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8
1,750001,male,26,200.0,97.0,20.0,101.0,40.5
2,750002,female,29,188.0,85.0,16.0,102.0,40.4
3,750003,female,39,172.0,73.0,20.0,107.0,40.6
4,750004,female,30,173.0,67.0,16.0,94.0,40.5


In [16]:
train=train.drop(["id"],axis=1)
train.head()

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [17]:
# Male--->1 , female ----> 0
train["Sex"]=np.where(train["Sex"]=="male",1,0)
test["Sex"]=np.where(test["Sex"]=="male",1,0)
print(train.head(),test.head())

   Sex  Age  Height  Weight  Duration  Heart_Rate  Body_Temp  Calories
0    1   36   189.0    82.0      26.0       101.0       41.0     150.0
1    0   64   163.0    60.0       8.0        85.0       39.7      34.0
2    0   51   161.0    64.0       7.0        84.0       39.8      29.0
3    1   20   192.0    90.0      25.0       105.0       40.7     140.0
4    0   38   166.0    61.0      25.0       102.0       40.6     146.0        id  Sex  Age  Height  Weight  Duration  Heart_Rate  Body_Temp
0  750000    1   45   177.0    81.0       7.0        87.0       39.8
1  750001    1   26   200.0    97.0      20.0       101.0       40.5
2  750002    0   29   188.0    85.0      16.0       102.0       40.4
3  750003    0   39   172.0    73.0      20.0       107.0       40.6
4  750004    0   30   173.0    67.0      16.0        94.0       40.5


In [18]:
y_train=train["Calories"]
train=train.drop(["Calories"],axis=1)
train.head()

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,1,36,189.0,82.0,26.0,101.0,41.0
1,0,64,163.0,60.0,8.0,85.0,39.7
2,0,51,161.0,64.0,7.0,84.0,39.8
3,1,20,192.0,90.0,25.0,105.0,40.7
4,0,38,166.0,61.0,25.0,102.0,40.6


In [19]:
y_train.head()

0    150.0
1     34.0
2     29.0
3    140.0
4    146.0
Name: Calories, dtype: float64

In [20]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [21]:
noise = 1.15

# Convert to numpy
train_tensor = torch.tensor(train.values, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float).unsqueeze(1)

train_numpy = train_tensor.numpy()
y_train_numpy = y_train_tensor.numpy()

# Use two separate scalers
X_scaler = StandardScaler()
y_scaler = StandardScaler()

train_np = X_scaler.fit_transform(train_numpy)
y_train_np = y_scaler.fit_transform(y_train_numpy)

# Convert back to torch tensors
train_tensor = torch.tensor(train_np, dtype=torch.float)
y_train_tensor = torch.tensor(y_train_np, dtype=torch.float)

# Prepare dataset and loaders
training_data = TensorDataset(noise * train_tensor, y_train_tensor)

val_ratio = 0.2
val_size = int(len(training_data) * val_ratio)
train_size = len(training_data) - val_size

training_data, val_data = random_split(training_data, [train_size, val_size])

train_loader = DataLoader(training_data, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=4)


In [22]:
import joblib
joblib.dump(X_scaler, 'X_scaler.pkl')
joblib.dump(y_scaler, 'y_scaler.pkl')

['y_scaler.pkl']

In [23]:
print(len(train),len(y_train))
print(train_tensor.shape,y_train_tensor.shape)
print(len(train_loader),len(val_loader))
for xb, yb in train_loader:
    print(f"Batch train X shape: {xb.shape}")
    print(f"Batch train y shape: {yb.shape}")
    break  # just one batch

750000 750000
torch.Size([750000, 7]) torch.Size([750000, 1])
18750 4688
Batch train X shape: torch.Size([32, 7])
Batch train y shape: torch.Size([32, 1])


#  Model Development

     Noise added
     Data normalized
     Momentum auto adapted
     Regularization auto integrated 
     Reduce Lr on plateau
     Dropping
     Early stopping
     Deep Network
     Batch Norm in training
     distributed dropout
     Grad_clip

 use nn.BatchNorm1d()

In [24]:
class calorie(nn.Module):
    def __init__(self):
        super(calorie,self).__init__() # Even if i override parent class still i can use parent class methods.
        self.net=nn.Sequential(
            nn.Linear(7,56),
            nn.BatchNorm1d(56),
            nn.ReLU(),
            
            nn.Linear(56,224),
            nn.BatchNorm1d(224),
            nn.ReLU(),
            
            nn.Linear(224,896),
            nn.BatchNorm1d(896),
            nn.ReLU(),
            
            nn.Linear(896,448),
            nn.BatchNorm1d(448),
            nn.ReLU(),
            
            nn.Dropout(.3),
            
            nn.Linear(448,112),
            nn.BatchNorm1d(112),
            nn.ReLU(),
            
            nn.Linear(112,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            
            nn.Dropout(.2),            
            
            nn.Linear(64,32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            
            nn.Linear(32,8),
            nn.BatchNorm1d(8),
            nn.ReLU(),
            
            nn.Linear(8,1),
        ) # Use of f(x)--> R-->G()---> R---> H()...makes it behave like a non linear regression

    def forward(self,x):
        return self.net(x)

model=calorie().to(device)

In [25]:
# Define loss, optimizer, scheduler
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

# Metrics
train_metric = R2Score().to(device)
val_metric = R2Score().to(device)

# Training setup
epochs = 100
patience = 5
best_val_r2 = float('-inf')
patience_counter = 0
best_model_state = None

# Training loop
for epoch in range(1, epochs + 1):
    model.train()
    train_metric.reset()
    total_train_loss = 0.0

    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch}/{epochs} [Training]", leave=False)
    for batch_idx, (x, y) in enumerate(train_loader_tqdm):
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output, y)
        loss.backward()
        nn.utils.clip_grad_norm(model.parameters(),max_norm=1.0)
        optimizer.step()

        train_metric.update(output, y)
        total_train_loss += loss.item()

        if (batch_idx + 1) % 1000 == 0:
            train_loader_tqdm.set_postfix(loss=loss.item())

    train_r2 = train_metric.compute()
    avg_train_loss = total_train_loss / len(train_loader)

    # Validation
    model.eval()
    val_metric.reset()
    total_val_loss = 0.0

    with torch.no_grad():
        val_loader_tqdm = tqdm(val_loader, desc=f"Epoch {epoch}/{epochs} [Validation]", leave=False)
        for x, y in val_loader_tqdm:
            x = x.to(device)
            y = y.to(device)
            preds = model(x)
            val_loss = loss_fn(preds, y)
            total_val_loss += val_loss.item()
            val_metric.update(preds, y)

    val_r2 = val_metric.compute()
    avg_val_loss = total_val_loss / len(val_loader)

    # Reduce LR scheduler
    scheduler.step(avg_val_loss)

    # Logging
    if epoch % 2 == 0 or epoch == 1:
        print(f"Epoch {epoch}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | "
              f"Train R2: {train_r2:.4f} | Val R2: {val_r2:.4f}")

    # Early stopping
    if val_r2 > best_val_r2:
        best_val_r2 = val_r2
        patience_counter = 0
        best_model_state = model.state_dict()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch}. Best Val R2: {best_val_r2:.4f}")
            if best_model_state is not None:
                torch.save(best_model_state, 'Calories_Prediction.pt')
                model.load_state_dict(best_model_state)
            break


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/100 [Training]:   0%|          | 0/18750 [00:00<?, ?it/s]/tmp/ipykernel_35/2499535233.py:32: FutureWarning: `torch.nn.utils.clip_grad_norm` is now deprecated in favor of `torch.nn.utils.clip_grad_norm_`.
  nn.utils.clip_grad_norm(model.parameters(),max_norm=1.0)


Epoch 1/100 | Train Loss: 0.0386 | Val Loss: 0.0085 | Train R2: 0.9614 | Val R2: 0.9915


Epoch 2/100 | Train Loss: 0.0180 | Val Loss: 0.0151 | Train R2: 0.9820 | Val R2: 0.9850


Epoch 4/100 | Train Loss: 0.0136 | Val Loss: 0.0107 | Train R2: 0.9864 | Val R2: 0.9893


Epoch 6/100 | Train Loss: 0.0105 | Val Loss: 0.0125 | Train R2: 0.9895 | Val R2: 0.9875

Early stopping at epoch 6. Best Val R2: 0.9915


In [26]:
test.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,1,45,177.0,81.0,7.0,87.0,39.8
1,750001,1,26,200.0,97.0,20.0,101.0,40.5
2,750002,0,29,188.0,85.0,16.0,102.0,40.4
3,750003,0,39,172.0,73.0,20.0,107.0,40.6
4,750004,0,30,173.0,67.0,16.0,94.0,40.5


In [27]:
ids=test["id"]
test=test.drop("id", axis=1)
test.head()

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,1,45,177.0,81.0,7.0,87.0,39.8
1,1,26,200.0,97.0,20.0,101.0,40.5
2,0,29,188.0,85.0,16.0,102.0,40.4
3,0,39,172.0,73.0,20.0,107.0,40.6
4,0,30,173.0,67.0,16.0,94.0,40.5


In [28]:
test_tensor=torch.tensor(test.values,dtype=torch.float)

test_numpy=test_tensor.numpy()

# Scaling
test_scaler_np=X_scaler.fit_transform(test_numpy)
test_tensor=torch.tensor(test_scaler_np, dtype= torch.float)

In [29]:
model.eval()
test_tensor=test_tensor.to(device)
with torch.no_grad():
    output=model(test_tensor)
    predictions=output.cpu().numpy()
    predictions=y_scaler.inverse_transform(predictions)
    print(predictions) # 2D

[[ 37.07574]
 [103.88339]
 [ 86.48013]
 ...
 [ 75.59096]
 [150.37173]
 [ 80.19188]]


In [30]:
submission=pd.DataFrame(
    {
        "id":ids,
        "Calories":predictions.flatten()
    }
)
submission.head()

,id,Calories
0,750000,37.075741
1,750001,103.883392
2,750002,86.480133
3,750003,116.782280
4,750004,79.270172


In [31]:
submission.to_csv("Predicted_Calories_Burnt.csv",index=False)